In [ ]:
# ============================================================================
# Abstract base class for noise models
# ============================================================================
class NoiseModel(ABC):
    """Base class for noise models"""
    
    @abstractmethod
    def apply(self, data: np.ndarray, **kwargs) -> np.ndarray:
        """
        Apply noise to the data
        
        Parameters
        ----------
        data : np.ndarray
            Input data array
        **kwargs : additional parameters for the noise model
        
        Returns
        -------
        np.ndarray
            Data with noise added
        """
        pass

# ============================================================================
# Concrete noise model implementations. More models can be added if necessary.
# ============================================================================
class PoissonNoise(NoiseModel):
    """Shot noise following Poisson statistics"""
    
    def apply(self, data: np.ndarray, scale: float = 1.0) -> np.ndarray:
        """
        Apply Poisson noise
        
        Parameters
        ----------
        data : np.ndarray
            Input data (should be non-negative)
        scale : float
            Scaling factor for intensity before applying Poisson
        
        Returns
        -------
        np.ndarray
            Data with Poisson noise
        """
        # Scale data, apply Poisson, scale back
        scaled = data * scale
        noisy = np.random.poisson(scaled)
        return noisy / scale


class GaussianNoise(NoiseModel):
    """Additive Gaussian (normal) noise"""
    
    def apply(self, data: np.ndarray, mean: float = 0.0, 
              sigma: float = 1.0) -> np.ndarray:
        """
        Apply Gaussian noise
        
        Parameters
        ----------
        data : np.ndarray
            Input data
        mean : float
            Mean of Gaussian distribution
        sigma : float
            Standard deviation of Gaussian distribution
        
        Returns
        -------
        np.ndarray
            Data with Gaussian noise added
        """
        noise = np.random.normal(mean, sigma, data.shape)
        return data + noise


class ReadoutNoise(NoiseModel):
    """Detector readout noise (Gaussian)"""
    
    def apply(self, data: np.ndarray, sigma: float = 5.0) -> np.ndarray:
        """
        Apply readout noise
        
        Parameters
        ----------
        data : np.ndarray
            Input data
        sigma : float
            Standard deviation of readout noise in counts
        
        Returns
        -------
        np.ndarray
            Data with readout noise added
        """
        noise = np.random.normal(0, sigma, data.shape)
        return data + noise


class DarkCurrentNoise(NoiseModel):
    """Dark current noise (Poisson-distributed)"""
    
    def apply(self, data: np.ndarray, dark_current: float = 1.0) -> np.ndarray:
        """
        Apply dark current noise
        
        Parameters
        ----------
        data : np.ndarray
            Input data
        dark_current : float
            Mean dark current in counts per pixel
        
        Returns
        -------
        np.ndarray
            Data with dark current noise added
        """
        dark = np.random.poisson(dark_current, data.shape)
        return data + dark


class SaltPepperNoise(NoiseModel):
    """Salt and pepper (impulse) noise"""
    
    def apply(self, data: np.ndarray, probability: float = 0.01,
              salt_value: Optional[float] = None,
              pepper_value: float = 0.0) -> np.ndarray:
        """
        Apply salt and pepper noise
        
        Parameters
        ----------
        data : np.ndarray
            Input data
        probability : float
            Probability of a pixel being affected (total for both salt and pepper)
        salt_value : float, optional
            Value for 'salt' pixels. If None, uses max of data
        pepper_value : float
            Value for 'pepper' pixels
        
        Returns
        -------
        np.ndarray
            Data with salt and pepper noise
        """
        noisy = data.copy()
        
        if salt_value is None:
            salt_value = np.max(data)
        
        # Salt noise
        salt_mask = np.random.random(data.shape) < (probability / 2)
        noisy[salt_mask] = salt_value
        
        # Pepper noise
        pepper_mask = np.random.random(data.shape) < (probability / 2)
        noisy[pepper_mask] = pepper_value
        
        return noisy


class CorrelatedNoise(NoiseModel):
    """Spatially correlated noise (low-frequency)"""
    
    def apply(self, data: np.ndarray, sigma: float = 1.0,
              correlation_length: float = 5.0) -> np.ndarray:
        """
        Apply correlated noise using Gaussian filtering
        
        Parameters
        ----------
        data : np.ndarray
            Input data
        sigma : float
            Amplitude of noise
        correlation_length : float
            Correlation length in pixels
        
        Returns
        -------
        np.ndarray
            Data with correlated noise added
        """
        from scipy.ndimage import gaussian_filter
        
        # Generate white noise
        white_noise = np.random.normal(0, sigma, data.shape)
        
        # Smooth to create correlations
        correlated = gaussian_filter(white_noise, sigma=correlation_length)
        
        return data + correlated


# ============================================================================
# Main function to add noise to datacube
# ============================================================================

def add_noise_to_datacube(
    datacube: py4DSTEM.DataCube,
    noise_models: list[tuple[NoiseModel, Dict[str, Any]]],
    seed: Optional[int] = None,
    clip_negative: bool = True
) -> py4DSTEM.DataCube:
    """
    Add noise to Q-space (diffraction patterns) in a 4DSTEM datacube
    
    Parameters
    ----------
    datacube : py4DSTEM.DataCube
        Input datacube
    noise_models : list of tuples
        List of (NoiseModel instance, parameters dict) to apply sequentially
    seed : int, optional
        Random seed for reproducibility
    clip_negative : bool
        Whether to clip negative values to zero after adding noise
    
    Returns
    -------
    py4DSTEM.DataCube
        New datacube with noise added
    
    Examples
    --------
    >>> # Single noise source
    >>> noisy = add_noise_to_datacube(
    ...     datacube,
    ...     [(PoissonNoise(), {'scale': 100})]
    ... )
    
    >>> # Multiple noise sources
    >>> noisy = add_noise_to_datacube(
    ...     datacube,
    ...     [
    ...         (PoissonNoise(), {'scale': 100}),
    ...         (ReadoutNoise(), {'sigma': 5}),
    ...         (DarkCurrentNoise(), {'dark_current': 2})
    ...     ],
    ...     seed=42
    ... )
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Copy the data
    noisy_data = datacube.data.copy().astype(float)
    
    # Get shape
    scan_i, scan_j, det_i, det_j = noisy_data.shape
    
    print(f"Adding noise to datacube of shape {noisy_data.shape}")
    
    # Apply each noise model sequentially to each diffraction pattern
    for noise_idx, (noise_model, params) in enumerate(noise_models):
        print(f"Applying {noise_model.__class__.__name__} with params {params}...")
        
        # Apply noise to each diffraction pattern in Q-space
        for i in range(scan_i):
            for j in range(scan_j):
                # Get diffraction pattern
                dp = noisy_data[i, j, :, :]
                
                # Apply noise
                noisy_dp = noise_model.apply(dp, **params)
                
                # Store back
                noisy_data[i, j, :, :] = noisy_dp
    
    # Clip negative values if requested
    if clip_negative:
        noisy_data = np.maximum(noisy_data, 0)
    
    # Create new datacube
    noisy_datacube = py4DSTEM.DataCube(data=noisy_data)
    
    # Copy calibration if it exists
    if hasattr(datacube, 'calibration'):
        noisy_datacube.calibration = datacube.calibration
    
    # Store noise information in metadata
    noisy_datacube.metadata['noise_applied'] = [
        {
            'model': model.__class__.__name__,
            'parameters': params
        }
        for model, params in noise_models
    ]
    
    print("Noise addition complete!")
    return noisy_datacube


# ============================================================================
# Convenience function for common noise combinations
# ============================================================================

def add_realistic_detector_noise(
    datacube: py4DSTEM.DataCube,
    dose_scale: float = 100,
    readout_sigma: float = 5.0,
    dark_current: float = 1.0,
    seed: Optional[int] = None
) -> py4DSTEM.DataCube:
    """
    Add realistic detector noise (Poisson + readout + dark current)
    
    Parameters
    ----------
    datacube : py4DSTEM.DataCube
        Input datacube
    dose_scale : float
        Scaling for shot noise (higher = more signal, less relative noise)
    readout_sigma : float
        Readout noise standard deviation in counts
    dark_current : float
        Mean dark current in counts per pixel
    seed : int, optional
        Random seed
    
    Returns
    -------
    py4DSTEM.DataCube
        Datacube with realistic noise
    """
    noise_models = [
        (PoissonNoise(), {'scale': dose_scale}),
        (DarkCurrentNoise(), {'dark_current': dark_current}),
        (ReadoutNoise(), {'sigma': readout_sigma})
    ]
    
    return add_noise_to_datacube(datacube, noise_models, seed=seed)

In [ ]:
# Example 1: Single noise source
noisy_dc = add_noise_to_datacube(
    datacube,
    [(PoissonNoise(), {'scale': 50})]
)

# Example 2: Multiple noise sources
noisy_dc = add_noise_to_datacube(
    datacube,
    [
        (PoissonNoise(), {'scale': 100}),
        (GaussianNoise(), {'sigma': 2}),
        (ReadoutNoise(), {'sigma': 5})
    ],
    seed=42
)

# Example 3: Realistic detector noise
noisy_dc = add_realistic_detector_noise(
    datacube,
    dose_scale=80,
    readout_sigma=3.5,
    dark_current=0.5
)

# Example 4: Custom noise combination
noisy_dc = add_noise_to_datacube(
    datacube,
    [
        (PoissonNoise(), {'scale': 100}),
        (CorrelatedNoise(), {'sigma': 1.0, 'correlation_length': 3}),
        (SaltPepperNoise(), {'probability': 0.001})
    ]
)